### Baseline модель для определения именованных сущностей по кейсу от Rutube.
Поскольку нам нужно распознать нестандартные NER, можно воспользоваться помощью языковых моделей, в данном случае - Bert.
Данные вы уже получили  - это разметка, сделанная на Толоке, она не идеальна, но это часть практической задачи, с которой можно столкнуться в реальности. 

Небольшое введение в NER https://habr.com/ru/companies/contentai/articles/449514/

In [ ]:
#!pip install transformers[torch] 
#!pip install datasets seqeval
#!pip install corus razdel 
#!pip install torch==2.0.0
#!pip install pandas

In [1]:
# считаем данные
import os 
import torch
import pandas as pd

batch_size = 32
num_train_epochs = 100
model_checkpoint = "cointegrated/rubert-tiny"
name_folder_and_model = 'ner_bert_100.bin'

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
is_cuda = torch.cuda.is_available()
print(is_cuda)
data = pd.read_csv("ner_data_train.csv")

True


In [2]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать, 
# удалить лишние '\' например, преобразовать из str в список dict-ов
import json
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

#### Оригинал туториала на медицинских данных можно посмотреть тут https://gist.github.com/avidale/cacf235aebeaaf4c578389e1146c3c57

In [3]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его тег (label) из разметки, чтобы потом предать в модель классификации токенов
from razdel import tokenize

def extract_labels(item):
    
    # воспользуемся удобным токенайзером из библиотеки razdel, 
    # она помимо разбиения на слова, сохраняет важные для нас числа - начало и конец слова в токенах
    
    raw_toks = list(tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

### Обработаем датасет и разобьем на трейн и тест

In [4]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for i, item in df.iterrows()]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

In [5]:
pd.options.display.max_colwidth = 300

#### Посмотрим на получившиеся теги
Подробнее почитать про BIO теги можно тут https://datascience.stackexchange.com/questions/63399/what-is-bio-tags-for-creating-custom-ner-named-entity-recognization

In [6]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Дата',
 'B-бренд',
 'B-вид спорта',
 'B-видеоигра',
 'B-команда',
 'B-лига',
 'B-локация',
 'B-модель',
 'B-название проекта',
 'B-организация',
 'B-персона',
 'B-сезон',
 'B-серия',
 'I-Дата',
 'I-бренд',
 'I-вид спорта',
 'I-видеоигра',
 'I-команда',
 'I-лига',
 'I-локация',
 'I-модель',
 'I-название проекта',
 'I-организация',
 'I-персона',
 'I-сезон',
 'I-серия']

In [7]:
from datasets import Dataset, DatasetDict

In [8]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 5137
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1285
    })
})

### Запустим модель RuBert-tiny - классический Bert, поверх которого навешен слой классификации токенов.

In [9]:
from transformers import AutoTokenizer 
from datasets import load_dataset, load_metric

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device='gpu')

In [10]:
example = ner_train[5]
print(example["tokens"])

['<', 'НАЗВАНИЕ', ':', '>', 'Московский', 'международный', 'фестиваль', 'мира', '=', '89', 'и', 'Стас', 'Намин', '.', '«', 'Главный', 'день', '»', '<', 'ОПИСАНИЕ', ':', '>', '12', 'и', '13', 'июня', '1989', 'года', 'на', 'Центральном', 'стадионе', 'имени', 'В', '.', 'И', '.', 'Ленина', 'состоялся', 'первый', 'международный', 'рок', 'фестиваль', '.', 'Концерт', 'собрал', 'свыше', '100', 'тысяч', 'зрителей', '.', 'Советская', 'молодежь', 'тогда', 'увидела', 'и', 'услышала', 'признанных', 'мировых', 'кумиров', ':', 'Scorpions', ',', 'Bon', 'Jovi', ',', 'Ozzy', 'Osbourne', ',', 'Cinderella', ',', 'Motley', 'Crue', '.', 'Приезд', 'западных', 'звезд', 'казался', 'ожившей', 'фантастикой', '.', 'Фестиваль', 'стал', 'не', 'только', 'музыкальной', ',', 'но', 'и', 'важной', 'политической', 'акцией', '.', 'Вся', 'мировая', 'пресса', 'писала', 'о', 'том', ',', 'что', 'СССР', 'и', 'США', ',', 'долгое', 'время', 'находившиеся', 'в', 'состоянии', '"', 'холодной', 'войны', '"', ',', 'наконец', 'то', 'с

In [11]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '<', 'Н', '##АЗ', '##В', '##АН', '##И', '##Е', ':', '>', 'Московский', 'между', '##народный', 'фестиваль', 'мира', '=', '89', 'и', 'С', '##тас', 'На', '##мин', '.', '«', 'Главный', 'день', '»', '<', 'О', '##П', '##И', '##С', '##АН', '##И', '##Е', ':', '>', '12', 'и', '13', 'июня', '1989', 'года', 'на', 'Центр', '##альном', 'стадион', '##е', 'имени', 'В', '.', 'И', '.', 'Ленина', 'состоялся', 'первый', 'между', '##народный', 'рок', 'фестиваль', '.', 'К', '##он', '##цер', '##т', 'со', '##брал', 'свыше', '100', 'тысяч', 'зрителей', '.', 'Советская', 'м', '##оло', '##де', '##ж', '##ь', 'тогда', 'у', '##вид', '##ела', 'и', 'у', '##сл', '##ыш', '##ала', 'признан', '##ных', 'мир', '##овых', 'к', '##уми', '##ров', ':', 'Sc', '##orp', '##ions', ',', 'Bon', 'Jovi', ',', 'Oz', '##zy', 'Os', '##bour', '##ne', ',', 'Ci', '##nder', '##ella', ',', 'Mot', '##ley', 'C', '##rue', '.', 'При', '##езд', 'запад', '##ных', 'зв', '##езд', 'каз', '##ался', 'о', '##жив', '##шей', 'ф', '##ант', '##аст'

In [12]:
len(example["tags"]), len(tokenized_input["input_ids"])

(170, 274)

In [13]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 1, 1, 1, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 11, 11, 12, 12, 13, 14, 15, 16, 17, 18, 19, 19, 19, 19, 19, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 29, 30, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 40, 41, 42, 43, 43, 43, 43, 44, 44, 45, 46, 47, 48, 49, 50, 51, 51, 51, 51, 51, 52, 53, 53, 53, 54, 55, 55, 55, 55, 56, 56, 57, 57, 58, 58, 58, 59, 60, 60, 60, 61, 62, 63, 64, 65, 65, 66, 66, 66, 67, 68, 68, 68, 69, 70, 70, 71, 71, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 77, 78, 78, 78, 78, 79, 80, 80, 80, 81, 82, 83, 84, 85, 86, 87, 88, 88, 89, 90, 90, 90, 91, 92, 92, 93, 93, 94, 94, 95, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 104, 105, 106, 106, 106, 107, 108, 109, 110, 110, 110, 111, 112, 113, 114, 115, 116, 117, 117, 117, 118, 118, 118, 118, 119, 120, 121, 121, 122, 123, 124, 124, 125, 125, 126, 126, 127, 127, 127, 128, 128, 129, 130, 130, 130, 130, 131, 132, 132, 133, 133, 134, 135, 136, 137, 137, 137, 138, 139, 139, 139, 140, 141, 142, 143, 144, 144, 144, 

In [14]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

274 274


#### У Bert свой собсвенный токенайзер, который разбивает слова на мелкие токены, поэтому нам нужно корректно сопоставить токены и соответсвующие им неры.

In [15]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [16]:
tokenize_and_align_labels(ner_data['train'][1:2])

{'input_ids': [[2, 32, 293, 16218, 8117, 20795, 8759, 14386, 30, 34, 16992, 12449, 19314, 9147, 705, 19062, 1211, 2179, 15349, 603, 32, 294, 3932, 8759, 3330, 20795, 8759, 14386, 30, 34, 294, 18147, 5972, 2386, 1928, 733, 22970, 613, 17565, 29484, 15479, 12122, 18398, 13334, 1, 16992, 12449, 19314, 9147, 705, 19062, 1211, 2179, 15349, 603, 18, 282, 3200, 860, 20727, 12029, 4375, 29463, 2013, 2262, 18, 294, 15735, 29137, 320, 18029, 18398, 24952, 753, 705, 11316, 4307, 329, 22067, 12427, 283, 4297, 292, 29172, 3330, 1928, 705, 13790, 294, 22797, 1768, 10203, 761, 13951, 3243, 2535, 2641, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [17]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5137 [00:00<?, ? examples/s]

Map:   0%|          | 0/1285 [00:00<?, ? examples/s]

#### Сохраняем словарик соотвествия тега и его индекса внутри модели

In [18]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# Специальный объект для удобного формирования батчей
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

### В качестве метрик возьмем precision, recall, accuracy, для этого воспользуемся специализированной под Ner задачу библиотеку seqeval

In [20]:
metric = load_metric("seqeval")

C:\Users\tommy\AppData\Local\Temp\ipykernel_20316\152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [21]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'бренд': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'локация': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [22]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [23]:
# что мы видим без дообучения модели
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 3.261301040649414,
 'eval_precision': 0.004467048653396953,
 'eval_recall': 0.05039981979952697,
 'eval_f1': 0.008206717588050284,
 'eval_accuracy': 0.03661999084593258,
 'eval_runtime': 12.2009,
 'eval_samples_per_second': 105.32,
 'eval_steps_per_second': 3.36}

In [24]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [25]:
# Для дообучения берта можно эксперементировать с заморозкой/разморозкой разных слоев, здесь мы оставим все слои размороженными 
# Для быстроты обучения можно заморозить всю бертовую часть, кроме классификатора, но тогда качесвто будет похуже
for param in model.parameters():
    param.requires_grad = True

In [26]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [27]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.729894,0.000000,0.000000,0.000000,0.869086
2,No log,0.611510,0.160550,0.007884,0.015030,0.871661
3,No log,0.575236,0.154555,0.024552,0.042373,0.876922
4,0.918000,0.551991,0.205763,0.039813,0.066717,0.878377
5,0.918000,0.535103,0.235742,0.060987,0.096904,0.879661
6,0.918000,0.520812,0.252919,0.064647,0.102973,0.880681
7,0.546600,0.508229,0.265664,0.077599,0.120113,0.882125
8,0.546600,0.498044,0.287173,0.112456,0.161622,0.883334
9,0.546600,0.485325,0.332393,0.126028,0.182761,0.886095
10,0.501600,0.475980,0.333952,0.162124,0.218280,0.887182


TrainOutput(global_step=16100, training_loss=0.35016645964628423, metrics={'train_runtime': 3433.2289, 'train_samples_per_second': 149.626, 'train_steps_per_second': 4.689, 'total_flos': 2979991782365598.0, 'train_loss': 0.35016645964628423, 'epoch': 100.0})

In [29]:
trainer.evaluate()

{'eval_loss': 0.43732374906539917,
 'eval_precision': 0.3868989626912717,
 'eval_recall': 0.3801666854375493,
 'eval_f1': 0.38350328059760846,
 'eval_accuracy': 0.8893660250136752,
 'eval_runtime': 4.9212,
 'eval_samples_per_second': 261.118,
 'eval_steps_per_second': 8.331,
 'epoch': 100.0}

In [30]:
# Посчитаем метрики на отложенном датасете

predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'Дата': {'precision': 0.5232876712328767,
  'recall': 0.6366666666666667,
  'f1': 0.5744360902255639,
  'number': 1200},
 'бренд': {'precision': 0.3333333333333333,
  'recall': 0.1420323325635104,
  'f1': 0.19919028340080971,
  'number': 866},
 'вид спорта': {'precision': 0.5294117647058824,
  'recall': 0.035856573705179286,
  'f1': 0.06716417910447763,
  'number': 251},
 'видеоигра': {'precision': 0.2336448598130841,
  'recall': 0.23458445040214476,
  'f1': 0.23411371237458195,
  'number': 746},
 'команда': {'precision': 0.3699421965317919,
  'recall': 0.1553398058252427,
  'f1': 0.21880341880341878,
  'number': 412},
 'лига': {'precision': 0.1484375,
  'recall': 0.1347517730496454,
  'f1': 0.1412639405204461,
  'number': 141},
 'локация': {'precision': 0.3888742420247825,
  'recall': 0.4908485856905158,
  'f1': 0.4339511621065019,
  'number': 3005},
 'модель': {'precision': 0.21294363256784968,
  'recall': 0.20901639344262296,
  'f1': 0.21096173733195447,
  'number': 488},
 'названи

In [31]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [32]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-Дата,B-бренд,B-вид спорта,B-видеоигра,B-команда,B-лига,B-локация,B-модель,B-название проекта,...,I-видеоигра,I-команда,I-лига,I-локация,I-модель,I-название проекта,I-организация,I-персона,I-сезон,I-серия
O,224254,429,121,5,242,44,31,1301,105,1069,...,236,17,74,451,165,1609,331,1200,0,0
B-Дата,261,862,0,0,0,0,0,18,0,1,...,0,0,1,0,1,6,0,0,0,0
B-бренд,571,0,131,0,8,0,0,10,59,17,...,4,0,0,4,2,7,3,3,0,0
B-вид спорта,194,1,0,10,0,4,0,4,0,12,...,0,0,8,0,0,9,0,0,0,0
B-видеоигра,417,0,3,0,215,0,0,2,2,42,...,54,0,0,0,0,4,0,0,0,0
B-команда,192,0,2,0,0,66,0,37,0,42,...,0,0,3,4,0,0,0,14,0,0
B-лига,91,2,0,0,0,3,25,2,0,5,...,0,0,12,0,0,1,0,0,0,0
B-локация,1003,9,0,0,2,13,0,1627,1,31,...,3,1,3,114,0,29,6,35,0,0
B-модель,221,0,71,0,6,0,0,0,120,1,...,13,0,1,0,42,0,2,4,0,0
B-название проекта,1798,1,12,0,17,7,3,27,6,1160,...,3,0,4,3,0,114,1,10,0,0


In [33]:
model.save_pretrained(name_folder_and_model)
tokenizer.save_pretrained(name_folder_and_model)

('ner_bert_100.bin\\tokenizer_config.json',
 'ner_bert_100.bin\\special_tokens_map.json',
 'ner_bert_100.bin\\vocab.txt',
 'ner_bert_100.bin\\added_tokens.json',
 'ner_bert_100.bin\\tokenizer.json')

### Посмотрим на результаты

In [34]:
# text = ' '.join(ner_train[25]['tokens'])
text = ner_train[25]['tokens']

In [35]:
import torch
from transformers import pipeline

pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device='cpu')

def predict_ner(text, tokenizer, model, pipe, verbose=True):
    tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    
    with torch.no_grad():
        pred = model(**tokens)
    # print(pred.logits.shape)
    indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
    token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
    labels = []
    for t, idx in zip(token_text, indices):
        if '##' not in t:
            labels.append(label_list[idx])
        if verbose:    
            print(f'{t:15s} {label_list[idx]:10s}')
    return text, pipe(text), labels

In [36]:
predict_ner(text, tokenizer, model, pipe)

[CLS]           O         
<               O         
Н               O         
##АЗ            O         
##В             O         
##АН            O         
##И             O         
##Е             O         
:               O         
>               O         
Как             O         
сделать         O         
ма              O         
##ску           O         
Глава           O         
##ря            O         
из              O         
фильма          O         
М               B-название проекта
##А             B-название проекта
##Й             B-название проекта
##О             B-название проекта
##Р             B-название проекта
Г               I-название проекта
##РО            I-название проекта
##М             I-название проекта
\               O         
М               O         
##ас            O         
##ка            O         
из              O         
му              O         
##ль            O         
##т             O         
##фильм         O 

(['<',
  'НАЗВАНИЕ',
  ':',
  '>',
  'Как',
  'сделать',
  'маску',
  'Главаря',
  'из',
  'фильма',
  'МАЙОР',
  'ГРОМ',
  '\\',
  'Маска',
  'из',
  'мультфильма',
  'Шайбу',
  ',',
  'Шайбу',
  '<',
  'ОПИСАНИЕ',
  ':',
  '>',
  'Маска',
  'Главаря',
  'купить',
  ':',
  '<',
  'LINK',
  '>',
  '54170275',
  'Силикон',
  'и',
  'пластик',
  '(',
  'промокод',
  'на',
  'скидку',
  ':',
  'planeta',
  ')',
  '<',
  'LINK',
  '>',
  'trade',
  '.',
  'ru',
  'Полиморфус',
  ':',
  'промокод',
  'на',
  'скидку',
  '(',
  'SP',
  ')',
  ':',
  '<',
  'LINK',
  '>',
  'Пластилин',
  'которым',
  'я',
  'леплю',
  ':',
  '<',
  'LINK',
  '>',
  'Художник',
  'artist_miller',
  ':',
  '<',
  'LINK',
  '>',
  'Маска',
  'из',
  'полиморфуса',
  ':',
  '<',
  'LINK',
  '>',
  'nVSA',
  'Группа',
  'вк',
  ':',
  '<',
  'LINK',
  '>',
  'instagram',
  ':',
  's',
  '.',
  'm',
  '.',
  'o',
  '.',
  't',
  '.',
  'r',
  '.',
  'i_planeta',
  'мой',
  'второй',
  'канал',
  ':',
  '<',
  'LIN

### Тестового датасета у вас пока нет, по которому будет считаться метрика на лидерборде, но прогоним для примера через нашу отложенную выборку, чтобы понять формат выходных данных.
ВАЖНО: в тестовом датасете у вас будет тест в том же формате, что он был в трейне 'video_info', в финальном сабмишене эту колонку и индексы менять нельзя, нужно будет только заполнить колонку 'entities_prediction'

In [48]:
#from tqdm.notebook import tqdm

submission = pd.DataFrame(columns=[['video_info', 'entities_prediction']])
submission['entities_prediction'] = submission['entities_prediction'].astype('object')
def sample_submission(text, tokenizer, model, pipe, submission):
    for i, elem in enumerate(ner_test[:3]):
        _, _, labels = predict_ner(elem['tokens'], tokenizer, model, pipe, verbose=False)
        submission.loc[i, 'video_info'] = ''.join(elem['tokens'])

        submission.loc[i, 'entities_prediction'] = [[label] for label in labels]
    return submission

In [49]:
result = sample_submission(text, tokenizer, model, pipe, submission)

In [50]:
result

,video_info,entities_prediction
0,<НАЗВАНИЕ:>СамыекрасивыеместаБоливии.Короткийвидеообзор.ДостопримечательностиБоливии.Слайд=Шоу.<ОПИСАНИЕ:>СамыекрасивыеместаБоливии.Короткийвидеообзор.ДостопримечательностиБоливии.СлайдШоу.#боливия#достопримечательности#видеообзор#южнаяамерика#столицы#лапас#слайдшоу#места#красивыеместа#история#в...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,<НАЗВАНИЕ:>ЭТОЖЕСТЬ!ОНХОТЕЛУБИТЬМЕНЯ!РЕЙДБАЗЫСГРОМИЛОЙ=LastDayonEarth:Survival<ОПИСАНИЕ:>ЭТОЖЕСТЬ!ОНХОТЕЛУБИТЬМЕНЯ!РЕЙДБАЗЫСГРОМИЛОЙСпасибозаЛайкиПодписку✅ВсевидеоLDOE►<LINK>SVVUGN_b7gKVRROVzzvHe0INSTAGRAM➤<LINK>ГРУППАВК➤<LINK>DISCORD➤<LINK>LastDayonEarth:Survival–мобильнаяигранавыживание!Скачат...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-видеоигра, I-видеоигра, I-видеоигра, I-видеоигра, I-видеоигра, I-видеоигра, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-видеоигр..."
2,"<НАЗВАНИЕ:>Оранжевый—хитсезона/OrangeIstheNewBlack(6сезон)Русскийтрейлер(субтитры)<ОПИСАНИЕ:>Премьера:27июля2018г.Канал:Netflix.Жанр:драма.Вролях:ТейлорШиллинг,УзоАдуба,ЛораПрипон,НаташаЛионн,ЛиаДеЛарияидр.ПайперЧепмен—обычнаяжительницаКоннектикута.Оналюбитсвоюблагополучнуюжизньивсееёприятныемел...","[O, O, O, O, O, B-название проекта, O, O, O, I-название проекта, B-название проекта, I-название проекта, I-название проекта, I-название проекта, I-название проекта, O, B-серия, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Дата, I-Дата, I-Дата, I-Дата, O, O, O, B-организация, O, O, O, O, O, O, O, O, ..."


In [53]:
result.to_csv('test.csv', index=False)

In [54]:
ner_test[0]

{'tokens': ['<',
  'НАЗВАНИЕ',
  ':',
  '>',
  'Самые',
  'красивые',
  'места',
  'Боливии',
  '.',
  'Короткий',
  'видеообзор',
  '.',
  'Достопримечательности',
  'Боливии',
  '.',
  'Слайд',
  '=',
  'Шоу',
  '.',
  '<',
  'ОПИСАНИЕ',
  ':',
  '>',
  'Самые',
  'красивые',
  'места',
  'Боливии',
  '.',
  'Короткий',
  'видеообзор',
  '.',
  'Достопримечательности',
  'Боливии',
  '.',
  'Слайд',
  'Шоу',
  '.',
  '#',
  'боливия',
  '#',
  'достопримечательности',
  '#',
  'видеообзор',
  '#',
  'южнаяамерика',
  '#',
  'столицы',
  '#',
  'лапас',
  '#',
  'слайдшоу',
  '#',
  'места',
  '#',
  'красивыеместа',
  '#',
  'история',
  '#',
  'видеообзор',
  '#',
  'короткийвидеообзор',
  '#',
  'жизньвболивии',
  '#',
  'отдыхмечты',
  '#',
  'путешествие',
  '#',
  'туризм',
  '#',
  'туризмболивия',
  '#',
  'боливиятуризм',
  '#',
  'сакаба',
  '#',
  'тринидад',
  '#',
  'сукре'],
 'tags': ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-локация',
  'O',
  'O',
  'O',
  'O

video_info             <НАЗВАНИЕ:>СамыекрасивыеместаБоливии.Короткийвидеообзор.ДостопримечательностиБоливии.Слайд=Шоу.<ОПИСАНИЕ:>СамыекрасивыеместаБоливии.Короткийвидеообзор.ДостопримечательностиБоливии.СлайдШоу.#боливия#достопримечательности#видеообзор#южнаяамерика#столицы#лапас#слайдшоу#места#красивыеместа#история#в...
entities_prediction                                                            [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]
Name: 0, dtype: object